In [ ]:
pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
pip install indoNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
# Import libraries
import pandas as pd
import time
import json
import csv
from datetime import datetime
import numpy as np
import random
import pickle
import joblib
import os

# Libraries for NLP Preprocessing
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from indoNLP.preprocessing import *

# Libraries for modeling & evaluation
from sklearn.utils import class_weight
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from math import *

# Libraries for Vizualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from wordcloud import WordCloud, STOPWORDS

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load Dataset

## Data Ganjar

In [ ]:
df = pd.read_csv('ganjar.csv')
df

,tweet
0,@Penyair_Berdiri Ganjar sudah mulai memantau I...
1,@Gus_Raharjo Ini dia nih jagoan gw yg akan men...
2,"Ganjar tuh tegas, nggak kayak yang cari-cari c..."
3,"Ganjar-Mahfud bukan hanya janji kosong, tetapi..."
4,Warga Jatim mendukung Ganjar-Mahfud karena mer...
...,...
1602,Keberhasilan di Jatim adalah bukti bahwa Pak G...
1603,Proses yang dijalani oleh Pak Ganjar adalah bu...
1604,Ganjar Pranowo adalah pemimpin yang mampu meng...
1605,Dukungan untuk Ganjar-Mahfud adalah ekspresi d...


In [ ]:
df.shape

(1607, 1)

#### Cleaning Text

In [ ]:
# Process Cleaning
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention @
    text = re.sub(r"[@][\w_-]+","", text)
    # remove link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['tweet_clean'] = df['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['tweet_clean'] = df['tweet_clean'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['tweet_clean'] = df['tweet_clean'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['tweet_clean'] = df['tweet_clean'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['tweet_clean'] = df['tweet_clean'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

# Remove symbol
def remove_symbol(text):
    text = ''.join(re.sub(r"[\!\@\#$\%\^\&\*\?\,\"\|\:]+", "", text))
    return text

df['tweet_clean'] = df['tweet_clean'].apply(remove_singl_char)
df.loc[0:1607,['tweet', 'tweet_clean']]

,tweet,tweet_clean
0,@Penyair_Berdiri Ganjar sudah mulai memantau I...,Ganjar sudah mulai memantau IKN dimana Ganjar ...
1,@Gus_Raharjo Ini dia nih jagoan gw yg akan men...,Ini dia nih jagoan gw yg akan menempati Istana...
2,"Ganjar tuh tegas, nggak kayak yang cari-cari c...",Ganjar tuh tegas nggak kayak yang caricari cel...
3,"Ganjar-Mahfud bukan hanya janji kosong, tetapi...",GanjarMahfud bukan hanya janji kosong tetapi r...
4,Warga Jatim mendukung Ganjar-Mahfud karena mer...,Warga Jatim mendukung GanjarMahfud karena mere...
...,...,...
1602,Keberhasilan di Jatim adalah bukti bahwa Pak G...,Keberhasilan di Jatim adalah bukti bahwa Pak G...
1603,Proses yang dijalani oleh Pak Ganjar adalah bu...,Proses yang dijalani oleh Pak Ganjar adalah bu...
1604,Ganjar Pranowo adalah pemimpin yang mampu meng...,Ganjar Pranowo adalah pemimpin yang mampu meng...
1605,Dukungan untuk Ganjar-Mahfud adalah ekspresi d...,Dukungan untuk GanjarMahfud adalah ekspresi da...


#### Case Folding

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df['case_folding'] = df['tweet_clean'].str.lower()

df.loc[0:1607,['tweet_clean', 'case_folding']]

,tweet_clean,case_folding
0,Ganjar sudah mulai memantau IKN dimana Ganjar ...,ganjar sudah mulai memantau ikn dimana ganjar ...
1,Ini dia nih jagoan gw yg akan menempati Istana...,ini dia nih jagoan gw yg akan menempati istana...
2,Ganjar tuh tegas nggak kayak yang caricari cel...,ganjar tuh tegas nggak kayak yang caricari cel...
3,GanjarMahfud bukan hanya janji kosong tetapi r...,ganjarmahfud bukan hanya janji kosong tetapi r...
4,Warga Jatim mendukung GanjarMahfud karena mere...,warga jatim mendukung ganjarmahfud karena mere...
...,...,...
1602,Keberhasilan di Jatim adalah bukti bahwa Pak G...,keberhasilan di jatim adalah bukti bahwa pak g...
1603,Proses yang dijalani oleh Pak Ganjar adalah bu...,proses yang dijalani oleh pak ganjar adalah bu...
1604,Ganjar Pranowo adalah pemimpin yang mampu meng...,ganjar pranowo adalah pemimpin yang mampu meng...
1605,Dukungan untuk GanjarMahfud adalah ekspresi da...,dukungan untuk ganjarmahfud adalah ekspresi da...


#### Tokenizing

In [ ]:
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# ------- Tokenizing ---------

# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tokenize'] = df['case_folding'].apply(word_tokenize_wrapper)
df.loc[0:1608,['case_folding', 'tokenize']]

,case_folding,tokenize
0,ganjar sudah mulai memantau ikn dimana ganjar ...,"[ganjar, sudah, mulai, memantau, ikn, dimana, ..."
1,ini dia nih jagoan gw yg akan menempati istana...,"[ini, dia, nih, jagoan, gw, yg, akan, menempat..."
2,ganjar tuh tegas nggak kayak yang caricari cel...,"[ganjar, tuh, tegas, nggak, kayak, yang, caric..."
3,ganjarmahfud bukan hanya janji kosong tetapi r...,"[ganjarmahfud, bukan, hanya, janji, kosong, te..."
4,warga jatim mendukung ganjarmahfud karena mere...,"[warga, jatim, mendukung, ganjarmahfud, karena..."
...,...,...
1602,keberhasilan di jatim adalah bukti bahwa pak g...,"[keberhasilan, di, jatim, adalah, bukti, bahwa..."
1603,proses yang dijalani oleh pak ganjar adalah bu...,"[proses, yang, dijalani, oleh, pak, ganjar, ad..."
1604,ganjar pranowo adalah pemimpin yang mampu meng...,"[ganjar, pranowo, adalah, pemimpin, yang, mamp..."
1605,dukungan untuk ganjarmahfud adalah ekspresi da...,"[dukungan, untuk, ganjarmahfud, adalah, ekspre..."


#### Normalisasi

In [ ]:
key_normalisasi = pd.read_excel('key_normalisasi.xlsx')
print(key_normalisasi.head())

key_normalisasi.shape

       singkat       hasil
0         abis       habis
1       accent     tekanan
2       accept      terima
3     accident  kecelakaan
4  achievement    prestasi


(3779, 2)

In [ ]:
normalizad_word = pd.read_excel("key_normalisasi.xlsx")
normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['normalisasi'] = df['tokenize'].apply(normalized_term)

df.loc[0:1607,['tokenize', 'normalisasi']]

,tokenize,normalisasi
0,"[ganjar, sudah, mulai, memantau, ikn, dimana, ...","[ganjar, sudah, mulai, memantau, ikn, dimana, ..."
1,"[ini, dia, nih, jagoan, gw, yg, akan, menempat...","[ini, dia, nih, jagoan, saya, yang, akan, mene..."
2,"[ganjar, tuh, tegas, nggak, kayak, yang, caric...","[ganjar, tuh, tegas, tidak, seperti, yang, car..."
3,"[ganjarmahfud, bukan, hanya, janji, kosong, te...","[ganjarmahfud, bukan, hanya, janji, kosong, te..."
4,"[warga, jatim, mendukung, ganjarmahfud, karena...","[warga, jatim, mendukung, ganjarmahfud, karena..."
...,...,...
1602,"[keberhasilan, di, jatim, adalah, bukti, bahwa...","[keberhasilan, di, jatim, adalah, bukti, bahwa..."
1603,"[proses, yang, dijalani, oleh, pak, ganjar, ad...","[proses, yang, dijalani, oleh, pak, ganjar, ad..."
1604,"[ganjar, pranowo, adalah, pemimpin, yang, mamp...","[ganjar, pranowo, adalah, pemimpin, yang, mamp..."
1605,"[dukungan, untuk, ganjarmahfud, adalah, ekspre...","[dukungan, untuk, ganjarmahfud, adalah, ekspre..."


#### Filtering (Stopword Removal)

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

list_stopwords = stopwords.words('indonesian')

In [ ]:
len(list_stopwords)

758

In [ ]:
list_stopwords[:10]

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir']

In [ ]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'boleh',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 'sekali',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'saja',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'begitu',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nggak',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah','gw','lu','lo','gtw','bukan',
                       'iyaa','si','ruarrr','itu','gue','dan','juga',
                       'cm','cmn','emg','mau','kali','jadi','begini',
                       'bila','nanti'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    #kata = [word for word in words if word not in list_stopwords]
    return [word for word in words if word not in list_stopwords]

df['stopwords'] = df['normalisasi'].apply(stopwords_removal)

df.loc[0:1607,['normalisasi', 'stopwords']]

,normalisasi,stopwords
0,"[ganjar, sudah, mulai, memantau, ikn, dimana, ...","[ganjar, memantau, ikn, dimana, ganjar, tuntas..."
1,"[ini, dia, nih, jagoan, saya, yang, akan, mene...","[jagoan, menempati, istana, kepresidenan, diik..."
2,"[ganjar, tuh, tegas, tidak, seperti, yang, car...","[ganjar, caricari, celah, hukum, lanjutgan, ga..."
3,"[ganjarmahfud, bukan, hanya, janji, kosong, te...","[ganjarmahfud, janji, kosong, realisasi, konkr..."
4,"[warga, jatim, mendukung, ganjarmahfud, karena...","[warga, jatim, mendukung, ganjarmahfud, berbic..."
...,...,...
1602,"[keberhasilan, di, jatim, adalah, bukti, bahwa...","[keberhasilan, jatim, bukti, ganjar, pemimpin,..."
1603,"[proses, yang, dijalani, oleh, pak, ganjar, ad...","[proses, dijalani, ganjar, bukti, kesuksesan, ..."
1604,"[ganjar, pranowo, adalah, pemimpin, yang, mamp...","[ganjar, pranowo, pemimpin, menggandeng, menca..."
1605,"[dukungan, untuk, ganjarmahfud, adalah, ekspre...","[dukungan, ganjarmahfud, ekspresi, warga, jati..."


#### Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd

# Create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stemmed wrapper function
def stemmed_wrapper(term):
    return stemmer.stem(term)

# Create a dictionary for unique terms
term_dict = {}
for document in df['stopwords']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

# Apply stemming to unique terms
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

# Apply stemmed term to the DataFrame
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['stemming'] = df['stopwords'].apply(get_stemmed_term)

df.loc[0:1607,['stopwords', 'stemming']]

,stopwords,stemming
0,"[ganjar, memantau, ikn, dimana, ganjar, tuntas...","[ganjar, pantau, ikn, mana, ganjar, tuntas, ik..."
1,"[jagoan, menempati, istana, kepresidenan, diik...","[jago, tempat, istana, presiden, diiknpak, gan..."
2,"[ganjar, caricari, celah, hukum, lanjutgan, ga...","[ganjar, caricari, celah, hukum, lanjutgan, ga..."
3,"[ganjarmahfud, janji, kosong, realisasi, konkr...","[ganjarmahfud, janji, kosong, realisasi, konkr..."
4,"[warga, jatim, mendukung, ganjarmahfud, berbic...","[warga, jatim, dukung, ganjarmahfud, bicara, a..."
...,...,...
1602,"[keberhasilan, jatim, bukti, ganjar, pemimpin,...","[hasil, jatim, bukti, ganjar, pimpin, ubah, pr..."
1603,"[proses, dijalani, ganjar, bukti, kesuksesan, ...","[proses, jalan, ganjar, bukti, sukses, jalan, ..."
1604,"[ganjar, pranowo, pemimpin, menggandeng, menca...","[ganjar, pranowo, pimpin, gandeng, capai, solu..."
1605,"[dukungan, ganjarmahfud, ekspresi, warga, jati...","[dukung, ganjarmahfud, ekspresi, warga, jatim,..."


In [ ]:
# Assume 'stemming' column contains lists of stemmed words
df['Clean'] = df['stemming'].apply(lambda x: ' '.join(x))

# Display the resulting DataFrame
df.head(10)

,tweet,tweet_clean,case_folding,tokenize,normalisasi,stopwords,stemming,Clean
0,@Penyair_Berdiri Ganjar sudah mulai memantau I...,Ganjar sudah mulai memantau IKN dimana Ganjar ...,ganjar sudah mulai memantau ikn dimana ganjar ...,"[ganjar, sudah, mulai, memantau, ikn, dimana, ...","[ganjar, sudah, mulai, memantau, ikn, dimana, ...","[ganjar, memantau, ikn, dimana, ganjar, tuntas...","[ganjar, pantau, ikn, mana, ganjar, tuntas, ik...",ganjar pantau ikn mana ganjar tuntas ikn ikn u...
1,@Gus_Raharjo Ini dia nih jagoan gw yg akan men...,Ini dia nih jagoan gw yg akan menempati Istana...,ini dia nih jagoan gw yg akan menempati istana...,"[ini, dia, nih, jagoan, gw, yg, akan, menempat...","[ini, dia, nih, jagoan, saya, yang, akan, mene...","[jagoan, menempati, istana, kepresidenan, diik...","[jago, tempat, istana, presiden, diiknpak, gan...",jago tempat istana presiden diiknpak ganjar ma...
2,"Ganjar tuh tegas, nggak kayak yang cari-cari c...",Ganjar tuh tegas nggak kayak yang caricari cel...,ganjar tuh tegas nggak kayak yang caricari cel...,"[ganjar, tuh, tegas, nggak, kayak, yang, caric...","[ganjar, tuh, tegas, tidak, seperti, yang, car...","[ganjar, caricari, celah, hukum, lanjutgan, ga...","[ganjar, caricari, celah, hukum, lanjutgan, ga...",ganjar caricari celah hukum lanjutgan ganjarsa...
3,"Ganjar-Mahfud bukan hanya janji kosong, tetapi...",GanjarMahfud bukan hanya janji kosong tetapi r...,ganjarmahfud bukan hanya janji kosong tetapi r...,"[ganjarmahfud, bukan, hanya, janji, kosong, te...","[ganjarmahfud, bukan, hanya, janji, kosong, te...","[ganjarmahfud, janji, kosong, realisasi, konkr...","[ganjarmahfud, janji, kosong, realisasi, konkr...",ganjarmahfud janji kosong realisasi konkrit pi...
4,Warga Jatim mendukung Ganjar-Mahfud karena mer...,Warga Jatim mendukung GanjarMahfud karena mere...,warga jatim mendukung ganjarmahfud karena mere...,"[warga, jatim, mendukung, ganjarmahfud, karena...","[warga, jatim, mendukung, ganjarmahfud, karena...","[warga, jatim, mendukung, ganjarmahfud, berbic...","[warga, jatim, dukung, ganjarmahfud, bicara, a...",warga jatim dukung ganjarmahfud bicara adil amal
5,Menggunakan KTP sebagai kunci akses semua laya...,Menggunakan KTP sebagai kunci akses semua laya...,menggunakan ktp sebagai kunci akses semua laya...,"[menggunakan, ktp, sebagai, kunci, akses, semu...","[menggunakan, ktp, sebagai, kunci, akses, semu...","[ktp, kunci, akses, layanan, pemerintah, brili...","[ktp, kunci, akses, layan, perintah, brilian, ...",ktp kunci akses layan perintah brilian ganjarm...
6,GANJAR MAHFUD melakukan konsolidasi tim di neg...,GANJAR MAHFUD melakukan konsolidasi tim di neg...,ganjar mahfud melakukan konsolidasi tim di neg...,"[ganjar, mahfud, melakukan, konsolidasi, tim, ...","[ganjar, mahfud, melakukan, konsolidasi, tim, ...","[ganjar, mahfud, konsolidasi, tim, negeri, jir...","[ganjar, mahfud, konsolidasi, tim, negeri, jir...",ganjar mahfud konsolidasi tim negeri jiran mal...
7,Pak Ganjar dan Pak Mahfud mengajarkan bahwa se...,Pak Ganjar dan Pak Mahfud mengajarkan bahwa se...,pak ganjar dan pak mahfud mengajarkan bahwa se...,"[pak, ganjar, dan, pak, mahfud, mengajarkan, b...","[pak, ganjar, dan, pak, mahfud, mengajarkan, b...","[ganjar, mahfud, mengajarkan, langkah, peluang...","[ganjar, mahfud, ajar, langkah, peluang, ajar,...",ganjar mahfud ajar langkah peluang ajar tumbuh...
8,Keberanian dan kesabaran Pak Mahfud MD dalam m...,Keberanian dan kesabaran Pak Mahfud MD dalam m...,keberanian dan kesabaran pak mahfud md dalam m...,"[keberanian, dan, kesabaran, pak, mahfud, md, ...","[keberanian, dan, kesabaran, pak, mahfud, md, ...","[keberanian, kesabaran, mahfud, menghadapi, ta...","[berani, sabar, mahfud, hadap, tantang, nilai,...",berani sabar mahfud hadap tantang nilai adopsi...
9,@Melihat_Indo Hanya Pak Ganjar capres yg terba...,Hanya Pak Ganjar capres yg terbaik,hanya pak ganjar capres yg terbaik,"[hanya, pak, ganjar, capres, yg, terbaik]","[hanya, pak, ganjar, capres, yang, terbaik]","[ganjar, capres,

In [ ]:
# Menyimpan DataFrame ke dalam file Excel
df.to_excel('text_preprocessing_ganjar.xlsx', index=False)

In [ ]:
data_save = df["Clean"]

In [ ]:
data_save

0       ganjar pantau ikn mana ganjar tuntas ikn ikn u...
1       jago tempat istana presiden diiknpak ganjar ma...
2       ganjar caricari celah hukum lanjutgan ganjarsa...
3       ganjarmahfud janji kosong realisasi konkrit pi...
4        warga jatim dukung ganjarmahfud bicara adil amal
                              ...                        
1602    hasil jatim bukti ganjar pimpin ubah proses pr...
1603          proses jalan ganjar bukti sukses jalan tuju
1604      ganjar pranowo pimpin gandeng capai solusi adil
1605    dukung ganjarmahfud ekspresi warga jatim ubah ...
1606    ganjar pimpin visi rancang program ktp sakti l...
Name: Clean, Length: 1607, dtype: object

In [ ]:
data_save.to_csv('ganjar_clean_fix_2.csv')

In [ ]:
data_save.to_csv('ganjar_clean_fix_2.xlsx')

In [ ]:
# Import libraries
import csv
import nltk
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary
import string
import os

In [ ]:
#load dataset
df1 = pd.read_csv('ganjar_clean_fix_2.xlsx')
df1

,Unnamed: 0,Clean
0,0,ganjar pantau ikn mana ganjar tuntas ikn ikn u...
1,1,jago tempat istana presiden diiknpak ganjar ma...
2,2,ganjar caricari celah hukum lanjutgan ganjarsa...
3,3,ganjarmahfud janji kosong realisasi konkrit pi...
4,4,warga jatim dukung ganjarmahfud bicara adil amal
...,...,...
1602,1602,hasil jatim bukti ganjar pimpin ubah proses pr...
1603,1603,proses jalan ganjar bukti sukses jalan tuju
1604,1604,ganjar pranowo pimpin gandeng capai solusi adil
1605,1605,dukung ganjarmahfud ekspresi warga jatim ubah ...


In [ ]:
df1 = df1.drop(columns=['Unnamed: 0'])
df1

,Clean
0,ganjar pantau ikn mana ganjar tuntas ikn ikn u...
1,jago tempat istana presiden diiknpak ganjar ma...
2,ganjar caricari celah hukum lanjutgan ganjarsa...
3,ganjarmahfud janji kosong realisasi konkrit pi...
4,warga jatim dukung ganjarmahfud bicara adil amal
...,...
1602,hasil jatim bukti ganjar pimpin ubah proses pr...
1603,proses jalan ganjar bukti sukses jalan tuju
1604,ganjar pranowo pimpin gandeng capai solusi adil
1605,dukung ganjarmahfud ekspresi warga jatim ubah ...


In [ ]:
df1.columns

Index(['Clean'], dtype='object')

# Labelisasi Data

In [ ]:
lexicon_positif_dict = {}
with open('positive.tsv', 'r', newline='', encoding='utf-8') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')

    # Skip header
    next(reader, None)

    for row in reader:
        if row[0] not in lexicon_positif_dict:
            lexicon_positif_dict[row[0]] = int(row[1])

lexicon_negatif_dict = {}
with open('negative.tsv', 'r', newline='', encoding='utf-8') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')

    # Skip header
    next(reader, None)

    for row in reader:
        if row[0] not in lexicon_negatif_dict:
            lexicon_negatif_dict[row[0]] = int(row[1])

def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text:
        if word in lexicon_positif_dict:
            score += lexicon_positif_dict[word]
        if word in lexicon_negatif_dict:
            score += lexicon_negatif_dict[word]
    sentimen = 'Positif' if score >= 0 else 'Negatif'
    return score, sentimen

In [ ]:
df1

,Clean
0,ganjar pantau ikn mana ganjar tuntas ikn ikn u...
1,jago tempat istana presiden diiknpak ganjar ma...
2,ganjar caricari celah hukum lanjutgan ganjarsa...
3,ganjarmahfud janji kosong realisasi konkrit pi...
4,warga jatim dukung ganjarmahfud bicara adil amal
...,...
1602,hasil jatim bukti ganjar pimpin ubah proses pr...
1603,proses jalan ganjar bukti sukses jalan tuju
1604,ganjar pranowo pimpin gandeng capai solusi adil
1605,dukung ganjarmahfud ekspresi warga jatim ubah ...


In [ ]:
df1['Tokenized 2'] = df1['Clean'].apply(lambda x: word_tokenize_wrapper(str(x)) if pd.notna(x) else '')

In [ ]:
df1

,Clean,Tokenized 2
0,ganjar pantau ikn mana ganjar tuntas ikn ikn u...,"[ganjar, pantau, ikn, mana, ganjar, tuntas, ik..."
1,jago tempat istana presiden diiknpak ganjar ma...,"[jago, tempat, istana, presiden, diiknpak, gan..."
2,ganjar caricari celah hukum lanjutgan ganjarsa...,"[ganjar, caricari, celah, hukum, lanjutgan, ga..."
3,ganjarmahfud janji kosong realisasi konkrit pi...,"[ganjarmahfud, janji, kosong, realisasi, konkr..."
4,warga jatim dukung ganjarmahfud bicara adil amal,"[warga, jatim, dukung, ganjarmahfud, bicara, a..."
...,...,...
1602,hasil jatim bukti ganjar pimpin ubah proses pr...,"[hasil, jatim, bukti, ganjar, pimpin, ubah, pr..."
1603,proses jalan ganjar bukti sukses jalan tuju,"[proses, jalan, ganjar, bukti, sukses, jalan, ..."
1604,ganjar pranowo pimpin gandeng capai solusi adil,"[ganjar, pranowo, pimpin, gandeng, capai, solu..."
1605,dukung ganjarmahfud ekspresi warga jatim ubah ...,"[dukung, ganjarmahfud, ekspresi, warga, jatim,..."


In [ ]:
# Perform sentiment analysis using the lexicon
results = df1['Tokenized 2'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df1['Polarity Score'] = results[0]
df1['polarity'] = results[1]

# Display relevant columns
print(df1['polarity'].value_counts()),df1.shape

polarity
Positif    1301
Negatif     306
Name: count, dtype: int64


(None, (1607, 4))

In [ ]:
df1

,Clean,Tokenized 2,Polarity Score,polarity
0,ganjar pantau ikn mana ganjar tuntas ikn ikn u...,"[ganjar, pantau, ikn, mana, ganjar, tuntas, ik...",13,Positif
1,jago tempat istana presiden diiknpak ganjar ma...,"[jago, tempat, istana, presiden, diiknpak, gan...",5,Positif
2,ganjar caricari celah hukum lanjutgan ganjarsa...,"[ganjar, caricari, celah, hukum, lanjutgan, ga...",-1,Negatif
3,ganjarmahfud janji kosong realisasi konkrit pi...,"[ganjarmahfud, janji, kosong, realisasi, konkr...",-9,Negatif
4,warga jatim dukung ganjarmahfud bicara adil amal,"[warga, jatim, dukung, ganjarmahfud, bicara, a...",9,Positif
...,...,...,...,...
1602,hasil jatim bukti ganjar pimpin ubah proses pr...,"[hasil, jatim, bukti, ganjar, pimpin, ubah, pr...",3,Positif
1603,proses jalan ganjar bukti sukses jalan tuju,"[proses, jalan, ganjar, bukti, sukses, jalan, ...",2,Positif
1604,ganjar pranowo pimpin gandeng capai solusi adil,"[ganjar, pranowo, pimpin, gandeng, capai, solu...",5,Positif
1605,dukung ganjarmahfud ekspresi warga jatim ubah ...,"[dukung, ganjarmahfud, ekspresi, warga, jatim,...",8,Positif


In [ ]:
df1.to_excel("Data Labelling Ganjar_2.xlsx")

# Data Prabowo

In [ ]:
df2 = pd.read_csv('prabowo.csv')
df2

,tweet
0,"Rabu 6 Desember 2023, Telah Selesai Acara Rapa..."
1,Tema debat tersebut merujuk pada visi nasional...
2,Debat terakhir pada tanggal 4 Februari 2024 de...
3,Selanjutnya tema debat keempat pada tanggal 21...
4,Tema debat ketiga pada tanggal 7 Januari 2024 ...
...,...
1756,"Prabowo-Gibran, didukung oleh ratusan mantan k..."
1757,Solidaritas positif: Mantan Kepala Desa Cirebo...
1758,Mantan Kepala Desa mendukung positif: Prabowo-...
1759,"Suara hati mantan Kades yang berkumpul, Prabow..."


### Cleaning Text

In [ ]:
# Process Cleaning
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention @
    text = re.sub(r"[@][\w_-]+","", text)
    # remove link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df2['tweet_clean'] = df2['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df2['tweet_clean'] = df2['tweet_clean'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df2['tweet_clean'] = df2['tweet_clean'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df2['tweet_clean'] = df2['tweet_clean'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df2['tweet_clean'] = df2['tweet_clean'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

# Remove symbol
def remove_symbol(text):
    text = ''.join(re.sub(r"[\!\@\#$\%\^\&\*\?\,\"\|\:]+", "", text))
    return text

df2['tweet_clean'] = df2['tweet_clean'].apply(remove_singl_char)
df2.loc[0:1761,['tweet', 'tweet_clean']]

,tweet,tweet_clean
0,"Rabu 6 Desember 2023, Telah Selesai Acara Rapa...",Rabu Desember Telah Selesai Acara Rapat Konsol...
1,Tema debat tersebut merujuk pada visi nasional...,Tema debat tersebut merujuk pada visi nasional...
2,Debat terakhir pada tanggal 4 Februari 2024 de...,Debat terakhir pada tanggal Februari dengan te...
3,Selanjutnya tema debat keempat pada tanggal 21...,Selanjutnya tema debat keempat pada tanggal Ja...
4,Tema debat ketiga pada tanggal 7 Januari 2024 ...,Tema debat ketiga pada tanggal Januari adalah ...
...,...,...
1756,"Prabowo-Gibran, didukung oleh ratusan mantan k...",PrabowoGibran didukung oleh ratusan mantan kad...
1757,Solidaritas positif: Mantan Kepala Desa Cirebo...,Solidaritas positif Mantan Kepala Desa Cirebon...
1758,Mantan Kepala Desa mendukung positif: Prabowo-...,Mantan Kepala Desa mendukung positif PrabowoGi...
1759,"Suara hati mantan Kades yang berkumpul, Prabow...",Suara hati mantan Kades yang berkumpul Prabowo...


### Case Folding

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df2['case_folding'] = df2['tweet_clean'].str.lower()

df2.loc[0:1761,['tweet_clean', 'case_folding']]

,tweet_clean,case_folding
0,Rabu Desember Telah Selesai Acara Rapat Konsol...,rabu desember telah selesai acara rapat konsol...
1,Tema debat tersebut merujuk pada visi nasional...,tema debat tersebut merujuk pada visi nasional...
2,Debat terakhir pada tanggal Februari dengan te...,debat terakhir pada tanggal februari dengan te...
3,Selanjutnya tema debat keempat pada tanggal Ja...,selanjutnya tema debat keempat pada tanggal ja...
4,Tema debat ketiga pada tanggal Januari adalah ...,tema debat ketiga pada tanggal januari adalah ...
...,...,...
1756,PrabowoGibran didukung oleh ratusan mantan kad...,prabowogibran didukung oleh ratusan mantan kad...
1757,Solidaritas positif Mantan Kepala Desa Cirebon...,solidaritas positif mantan kepala desa cirebon...
1758,Mantan Kepala Desa mendukung positif PrabowoGi...,mantan kepala desa mendukung positif prabowogi...
1759,Suara hati mantan Kades yang berkumpul Prabowo...,suara hati mantan kades yang berkumpul prabowo...


### Tokenizing

In [ ]:
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# ------- Tokenizing ---------

# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df2['tokenize'] = df2['case_folding'].apply(word_tokenize_wrapper)
df2.loc[0:1761,['case_folding', 'tokenize']]

,case_folding,tokenize
0,rabu desember telah selesai acara rapat konsol...,"[rabu, desember, telah, selesai, acara, rapat,..."
1,tema debat tersebut merujuk pada visi nasional...,"[tema, debat, tersebut, merujuk, pada, visi, n..."
2,debat terakhir pada tanggal februari dengan te...,"[debat, terakhir, pada, tanggal, februari, den..."
3,selanjutnya tema debat keempat pada tanggal ja...,"[selanjutnya, tema, debat, keempat, pada, tang..."
4,tema debat ketiga pada tanggal januari adalah ...,"[tema, debat, ketiga, pada, tanggal, januari, ..."
...,...,...
1756,prabowogibran didukung oleh ratusan mantan kad...,"[prabowogibran, didukung, oleh, ratusan, manta..."
1757,solidaritas positif mantan kepala desa cirebon...,"[solidaritas, positif, mantan, kepala, desa, c..."
1758,mantan kepala desa mendukung positif prabowogi...,"[mantan, kepala, desa, mendukung, positif, pra..."
1759,suara hati mantan kades yang berkumpul prabowo...,"[suara, hati, mantan, kades, yang, berkumpul, ..."


### Normalisasi

In [ ]:
key_norm = pd.read_csv('key_normalisasi.csv')
print(key_norm.head())

key_norm.shape

       singkat       hasil
0         abis       habis
1       accent     tekanan
2       accept      terima
3     accident  kecelakaan
4  achievement    prestasi


(3779, 2)

In [ ]:
normalizad_word = pd.read_csv("key_normalisasi.csv")
normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df2['normalisasi'] = df2['tokenize'].apply(normalized_term)

df2.loc[0:1761,['tokenize', 'normalisasi']]

,tokenize,normalisasi
0,"[rabu, desember, telah, selesai, acara, rapat,...","[rabu, desember, telah, selesai, acara, rapat,..."
1,"[tema, debat, tersebut, merujuk, pada, visi, n...","[tema, debat, tersebut, merujuk, pada, visi, n..."
2,"[debat, terakhir, pada, tanggal, februari, den...","[debat, terakhir, pada, tanggal, februari, den..."
3,"[selanjutnya, tema, debat, keempat, pada, tang...","[selanjutnya, tema, debat, keempat, pada, tang..."
4,"[tema, debat, ketiga, pada, tanggal, januari, ...","[tema, debat, ketiga, pada, tanggal, januari, ..."
...,...,...
1756,"[prabowogibran, didukung, oleh, ratusan, manta...","[prabowogibran, didukung, oleh, ratusan, manta..."
1757,"[solidaritas, positif, mantan, kepala, desa, c...","[solidaritas, positif, mantan, kepala, desa, c..."
1758,"[mantan, kepala, desa, mendukung, positif, pra...","[mantan, kepala, desa, mendukung, positif, pra..."
1759,"[suara, hati, mantan, kades, yang, berkumpul, ...","[suara, hati, mantan, kades, yang, berkumpul, ..."


### Filtering (Stopword Removal)

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

list_stopwords = stopwords.words('indonesian')

In [ ]:
len(list_stopwords)

758

In [ ]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'boleh',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 'sekali',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'saja',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'begitu',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nggak',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah','gw','lu','lo','gtw','bukan',
                       'iyaa','si','ruarrr','itu','gue','dan','juga',
                       'cm','cmn','emg','mau','kali','jadi','begini',
                       'bila','nanti'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    #kata = [word for word in words if word not in list_stopwords]
    return [word for word in words if word not in list_stopwords]

df2['stopwords'] = df2['normalisasi'].apply(stopwords_removal)

df2.loc[0:1761,['normalisasi', 'stopwords']]

,normalisasi,stopwords
0,"[rabu, desember, telah, selesai, acara, rapat,...","[rabu, desember, selesai, acara, rapat, konsol..."
1,"[tema, debat, tersebut, merujuk, pada, visi, n...","[tema, debat, merujuk, visi, nasional, tercant..."
2,"[debat, terakhir, pada, tanggal, februari, den...","[debat, tanggal, februari, tema, teknologi, in..."
3,"[selanjutnya, tema, debat, keempat, pada, tang...","[tema, debat, keempat, tanggal, januari, perih..."
4,"[tema, debat, ketiga, pada, tanggal, januari, ...","[tema, debat, ketiga, tanggal, januari, ekonom..."
...,...,...
1756,"[prabowogibran, didukung, oleh, ratusan, manta...","[prabowogibran, didukung, ratusan, mantan, kad..."
1757,"[solidaritas, positif, mantan, kepala, desa, c...","[solidaritas, positif, mantan, kepala, desa, c..."
1758,"[mantan, kepala, desa, mendukung, positif, pra...","[mantan, kepala, desa, mendukung, positif, pra..."
1759,"[suara, hati, mantan, kades, yang, berkumpul, ...","[suara, hati, mantan, kades, berkumpul, prabow..."


### Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd

# Create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stemmed wrapper function
def stemmed_wrapper(term):
    return stemmer.stem(term)

# Create a dictionary for unique terms
term_dict = {}
for document in df2['stopwords']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

# Apply stemming to unique terms
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

# Apply stemmed term to the DataFrame
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df2['stemming'] = df2['stopwords'].apply(get_stemmed_term)

df2.loc[0:1761,['stopwords', 'stemming']]

,stopwords,stemming
0,"[rabu, desember, selesai, acara, rapat, konsol...","[rabu, desember, selesai, acara, rapat, konsol..."
1,"[tema, debat, merujuk, visi, nasional, tercant...","[tema, debat, rujuk, visi, nasional, cantum, b..."
2,"[debat, tanggal, februari, tema, teknologi, in...","[debat, tanggal, februari, tema, teknologi, in..."
3,"[tema, debat, keempat, tanggal, januari, perih...","[tema, debat, empat, tanggal, januari, perihal..."
4,"[tema, debat, ketiga, tanggal, januari, ekonom...","[tema, debat, tiga, tanggal, januari, ekonomi,..."
...,...,...
1756,"[prabowogibran, didukung, ratusan, mantan, kad...","[prabowogibran, dukung, ratus, mantan, kades, ..."
1757,"[solidaritas, positif, mantan, kepala, desa, c...","[solidaritas, positif, mantan, kepala, desa, c..."
1758,"[mantan, kepala, desa, mendukung, positif, pra...","[mantan, kepala, desa, dukung, positif, prabow..."
1759,"[suara, hati, mantan, kades, berkumpul, prabow...","[suara, hati, mantan, kades, kumpul, prabowogi..."


In [ ]:
# Assume 'stemming' column contains lists of stemmed words
df2['Clean'] = df2['stemming'].apply(lambda x: ' '.join(x))

# Display the resulting DataFrame
df2.head(10)

,tweet,tweet_clean,case_folding,tokenize,normalisasi,stopwords,stemming,Clean
0,"Rabu 6 Desember 2023, Telah Selesai Acara Rapa...",Rabu Desember Telah Selesai Acara Rapat Konsol...,rabu desember telah selesai acara rapat konsol...,"[rabu, desember, telah, selesai, acara, rapat,...","[rabu, desember, telah, selesai, acara, rapat,...","[rabu, desember, selesai, acara, rapat, konsol...","[rabu, desember, selesai, acara, rapat, konsol...",rabu desember selesai acara rapat konsolidasi ...
1,Tema debat tersebut merujuk pada visi nasional...,Tema debat tersebut merujuk pada visi nasional...,tema debat tersebut merujuk pada visi nasional...,"[tema, debat, tersebut, merujuk, pada, visi, n...","[tema, debat, tersebut, merujuk, pada, visi, n...","[tema, debat, merujuk, visi, nasional, tercant...","[tema, debat, rujuk, visi, nasional, cantum, b...",tema debat rujuk visi nasional cantum buka uud...
2,Debat terakhir pada tanggal 4 Februari 2024 de...,Debat terakhir pada tanggal Februari dengan te...,debat terakhir pada tanggal februari dengan te...,"[debat, terakhir, pada, tanggal, februari, den...","[debat, terakhir, pada, tanggal, februari, den...","[debat, tanggal, februari, tema, teknologi, in...","[debat, tanggal, februari, tema, teknologi, in...",debat tanggal februari tema teknologi informas...
3,Selanjutnya tema debat keempat pada tanggal 21...,Selanjutnya tema debat keempat pada tanggal Ja...,selanjutnya tema debat keempat pada tanggal ja...,"[selanjutnya, tema, debat, keempat, pada, tang...","[selanjutnya, tema, debat, keempat, pada, tang...","[tema, debat, keempat, tanggal, januari, perih...","[tema, debat, empat, tanggal, januari, perihal...",tema debat empat tanggal januari perihal energ...
4,Tema debat ketiga pada tanggal 7 Januari 2024 ...,Tema debat ketiga pada tanggal Januari adalah ...,tema debat ketiga pada tanggal januari adalah ...,"[tema, debat, ketiga, pada, tanggal, januari, ...","[tema, debat, ketiga, pada, tanggal, januari, ...","[tema, debat, ketiga, tanggal, januari, ekonom...","[tema, debat, tiga, tanggal, januari, ekonomi,...",tema debat tiga tanggal januari ekonomi rakyat...
5,Pada debat kedua yang dijadwalkan pada tanggal...,Pada debat kedua yang dijadwalkan pada tanggal...,pada debat kedua yang dijadwalkan pada tanggal...,"[pada, debat, kedua, yang, dijadwalkan, pada, ...","[pada, debat, kedua, yang, dijadwalkan, pada, ...","[debat, dijadwalkan, tanggal, desember, mengus...","[debat, jadwal, tanggal, desember, usung, tema...",debat jadwal tanggal desember usung tema tahan...
6,Debat pertama di Kantor KPU pada tanggal 12 De...,Debat pertama di Kantor KPU pada tanggal Desem...,debat pertama di kantor kpu pada tanggal desem...,"[debat, pertama, di, kantor, kpu, pada, tangga...","[debat, pertama, di, kantor, kpu, pada, tangga...","[debat, kantor, kpu, tanggal, desember, temany...","[debat, kantor, kpu, tanggal, desember, tema, ...",debat kantor kpu tanggal desember tema kait hu...
7,KPU RI telah menetapkan debat Pilpres 2024 seb...,KPU RI telah menetapkan debat Pilpres sebanyak...,kpu ri telah menetapkan debat pilpres sebanyak...,"[kpu, ri, telah, menetapkan, debat, pilpres, s...","[kpu, ri, telah, menetapkan, debat, pilpres, s...","[kpu, ri, menetapkan, debat, pilpres, calon, p...","[kpu, ri, tetap, debat, pilpres, calon, presid...",kpu ri tetap debat pilpres calon presiden calo...
8,agar format debat hanya berupa pemaparan dan p...,agar format debat hanya berupa pemaparan dan p...,agar format debat hanya berupa pemaparan dan p...,"[agar, format, debat, hanya, berupa, pemaparan...","[agar, format, debat, hanya, berupa, pemaparan...","[format, debat, pemaparan, pendalaman, dokumen...","[format, debat, papar, dalam, dokumen, visimis...",format debat papar dalam dokumen visimisi prab...
9,"Menurut dia, pada FGD tanggal 29 November 2023...",Menurut dia pada FGD tanggal November di KPU T...,menurut dia pada fgd tanggal november di kpu t...,"[menurut, dia, pada, fgd, tanggal, november, d...","[menurut, dia, pada, fg

In [ ]:
# Menyimpan DataFrame ke dalam file Excel
df2.to_excel('text_preprocessing_prabowo.xlsx', index=False)

In [ ]:
data_save = df2["Clean"]

In [ ]:
data_save

0       rabu desember selesai acara rapat konsolidasi ...
1       tema debat rujuk visi nasional cantum buka uud...
2       debat tanggal februari tema teknologi informas...
3       tema debat empat tanggal januari perihal energ...
4       tema debat tiga tanggal januari ekonomi rakyat...
                              ...                        
1756    prabowogibran dukung ratus mantan kades cirebo...
1757    solidaritas positif mantan kepala desa cirebon...
1758    mantan kepala desa dukung positif prabowogibra...
1759    suara hati mantan kades kumpul prabowogibran s...
1760    misi entas miskin prabowogibran dorong kuat de...
Name: Clean, Length: 1761, dtype: object

In [ ]:
data_save.to_csv('prabowo_clean_fix_2.csv')

In [ ]:
# Import libraries
import csv
import nltk
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary
import string
import os

In [ ]:
#load dataset
df3 = pd.read_csv('prabowo_clean_fix_2.csv')
df3

,Unnamed: 0,Clean
0,0,rabu desember selesai acara rapat konsolidasi ...
1,1,tema debat rujuk visi nasional cantum buka uud...
2,2,debat tanggal februari tema teknologi informas...
3,3,tema debat empat tanggal januari perihal energ...
4,4,tema debat tiga tanggal januari ekonomi rakyat...
...,...,...
1756,1756,prabowogibran dukung ratus mantan kades cirebo...
1757,1757,solidaritas positif mantan kepala desa cirebon...
1758,1758,mantan kepala desa dukung positif prabowogibra...
1759,1759,suara hati mantan kades kumpul prabowogibran s...


In [ ]:
df3 = df3.drop(columns=['Unnamed: 0'])
df3

,Clean
0,rabu desember selesai acara rapat konsolidasi ...
1,tema debat rujuk visi nasional cantum buka uud...
2,debat tanggal februari tema teknologi informas...
3,tema debat empat tanggal januari perihal energ...
4,tema debat tiga tanggal januari ekonomi rakyat...
...,...
1756,prabowogibran dukung ratus mantan kades cirebo...
1757,solidaritas positif mantan kepala desa cirebon...
1758,mantan kepala desa dukung positif prabowogibra...
1759,suara hati mantan kades kumpul prabowogibran s...


In [ ]:
df3.columns

Index(['Clean'], dtype='object')

# Labelisasi Data

In [ ]:
lexicon_positif_dict = {}
with open('positive.tsv', 'r', newline='', encoding='utf-8') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')

    # Skip header
    next(reader, None)

    for row in reader:
        if row[0] not in lexicon_positif_dict:
            lexicon_positif_dict[row[0]] = int(row[1])

lexicon_negatif_dict = {}
with open('negative.tsv', 'r', newline='', encoding='utf-8') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')

    # Skip header
    next(reader, None)

    for row in reader:
        if row[0] not in lexicon_negatif_dict:
            lexicon_negatif_dict[row[0]] = int(row[1])

def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text:
        if word in lexicon_positif_dict:
            score += lexicon_positif_dict[word]
        if word in lexicon_negatif_dict:
            score += lexicon_negatif_dict[word]
    sentimen = 'Positif' if score >= 0 else 'Negatif'
    return score, sentimen

In [ ]:
df3

,Clean
0,rabu desember selesai acara rapat konsolidasi ...
1,tema debat rujuk visi nasional cantum buka uud...
2,debat tanggal februari tema teknologi informas...
3,tema debat empat tanggal januari perihal energ...
4,tema debat tiga tanggal januari ekonomi rakyat...
...,...
1756,prabowogibran dukung ratus mantan kades cirebo...
1757,solidaritas positif mantan kepala desa cirebon...
1758,mantan kepala desa dukung positif prabowogibra...
1759,suara hati mantan kades kumpul prabowogibran s...


In [ ]:
df3['Tokenized 2'] = df3['Clean'].apply(lambda x: word_tokenize_wrapper(str(x)) if pd.notna(x) else '')

In [ ]:
df3

,Clean,Tokenized 2
0,rabu desember selesai acara rapat konsolidasi ...,"[rabu, desember, selesai, acara, rapat, konsol..."
1,tema debat rujuk visi nasional cantum buka uud...,"[tema, debat, rujuk, visi, nasional, cantum, b..."
2,debat tanggal februari tema teknologi informas...,"[debat, tanggal, februari, tema, teknologi, in..."
3,tema debat empat tanggal januari perihal energ...,"[tema, debat, empat, tanggal, januari, perihal..."
4,tema debat tiga tanggal januari ekonomi rakyat...,"[tema, debat, tiga, tanggal, januari, ekonomi,..."
...,...,...
1756,prabowogibran dukung ratus mantan kades cirebo...,"[prabowogibran, dukung, ratus, mantan, kades, ..."
1757,solidaritas positif mantan kepala desa cirebon...,"[solidaritas, positif, mantan, kepala, desa, c..."
1758,mantan kepala desa dukung positif prabowogibra...,"[mantan, kepala, desa, dukung, positif, prabow..."
1759,suara hati mantan kades kumpul prabowogibran s...,"[suara, hati, mantan, kades, kumpul, prabowogi..."


In [ ]:
# Perform sentiment analysis using the lexicon
results = df3['Tokenized 2'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df3['Polarity Score'] = results[0]
df3['polarity'] = results[1]

# Display relevant columns
print(df3['polarity'].value_counts()),df3.shape

polarity
Positif    951
Negatif    810
Name: count, dtype: int64


(None, (1761, 4))

In [ ]:
df3.to_excel("Data Labelling Prabowo_2.xlsx")

# Data Anies

### Load Dataset

In [ ]:
df4 = pd.read_csv('anies.csv')
df4

,tweet
0,@fairusizzudin @imanlagi Iyaa.. yg ditanya itu...
1,Si Marco Karundeng di kira saya Hebat ternyata...
2,@JamalBoegis @Abdulla55556212 Seneng lihat pak...
3,@aniesbaswedan Ruarrr biasa Capres pilihan Nom...
4,"prabowo kenapa harus sama gibran, mau dukung a..."
...,...
1602,Pilpres 2014: pilih Prabowo-Hatta karena tidak...
1603,@mediaindonesia Infrastuktur pengolahan ikan...
1604,"@mediaindonesia Anies Baswedan, berusaha men..."
1605,@yusufagro Sorry ya.. Lebih sexy kesalahan a...


### Cleaning Text

In [ ]:
# Process Cleaning
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention @
    text = re.sub(r"[@][\w_-]+","", text)
    # remove link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df4['tweet_clean'] = df4['tweet'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df4['tweet_clean'] = df4['tweet_clean'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df4['tweet_clean'] = df4['tweet_clean'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df4['tweet_clean'] = df4['tweet_clean'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df4['tweet_clean'] = df4['tweet_clean'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

# Remove symbol
def remove_symbol(text):
    text = ''.join(re.sub(r"[\!\@\#$\%\^\&\*\?\,\"\|\:]+", "", text))
    return text

df4['tweet_clean'] = df4['tweet_clean'].apply(remove_singl_char)
df4.loc[0:1607,['tweet', 'tweet_clean']]

,tweet,tweet_clean
0,@fairusizzudin @imanlagi Iyaa.. yg ditanya itu...,Iyaa yg ditanya itu lagi itu lagi yg nanya jg ...
1,Si Marco Karundeng di kira saya Hebat ternyata...,Si Marco Karundeng di kira saya Hebat ternyata...
2,@JamalBoegis @Abdulla55556212 Seneng lihat pak...,Seneng lihat pak Anies seneng
3,@aniesbaswedan Ruarrr biasa Capres pilihan Nom...,Ruarrr biasa Capres pilihan Nomor
4,"prabowo kenapa harus sama gibran, mau dukung a...",prabowo kenapa harus sama gibran mau dukung an...
...,...,...
1602,Pilpres 2014: pilih Prabowo-Hatta karena tidak...,Pilpres pilih PrabowoHatta karena tidak suka J...
1603,@mediaindonesia Infrastuktur pengolahan ikan...,Infrastuktur pengolahan ikan hal yg penting un...
1604,"@mediaindonesia Anies Baswedan, berusaha men...",Anies Baswedan berusaha menstabilkan dan menai...
1605,@yusufagro Sorry ya.. Lebih sexy kesalahan a...,Sorry ya Lebih sexy kesalahan asam sulfat dari...


### Case Folding

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df4['case_folding'] = df4['tweet_clean'].str.lower()

df4.loc[0:1607,['tweet_clean', 'case_folding']]

,tweet_clean,case_folding
0,Iyaa yg ditanya itu lagi itu lagi yg nanya jg ...,iyaa yg ditanya itu lagi itu lagi yg nanya jg ...
1,Si Marco Karundeng di kira saya Hebat ternyata...,si marco karundeng di kira saya hebat ternyata...
2,Seneng lihat pak Anies seneng,seneng lihat pak anies seneng
3,Ruarrr biasa Capres pilihan Nomor,ruarrr biasa capres pilihan nomor
4,prabowo kenapa harus sama gibran mau dukung an...,prabowo kenapa harus sama gibran mau dukung an...
...,...,...
1602,Pilpres pilih PrabowoHatta karena tidak suka J...,pilpres pilih prabowohatta karena tidak suka j...
1603,Infrastuktur pengolahan ikan hal yg penting un...,infrastuktur pengolahan ikan hal yg penting un...
1604,Anies Baswedan berusaha menstabilkan dan menai...,anies baswedan berusaha menstabilkan dan menai...
1605,Sorry ya Lebih sexy kesalahan asam sulfat dari...,sorry ya lebih sexy kesalahan asam sulfat dari...


### Tokenizing

In [ ]:
# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# ------- Tokenizing ---------

# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df4['tokenize'] = df4['case_folding'].apply(word_tokenize_wrapper)
df4.loc[0:1607,['case_folding', 'tokenize']]

,case_folding,tokenize
0,iyaa yg ditanya itu lagi itu lagi yg nanya jg ...,"[iyaa, yg, ditanya, itu, lagi, itu, lagi, yg, ..."
1,si marco karundeng di kira saya hebat ternyata...,"[si, marco, karundeng, di, kira, saya, hebat, ..."
2,seneng lihat pak anies seneng,"[seneng, lihat, pak, anies, seneng]"
3,ruarrr biasa capres pilihan nomor,"[ruarrr, biasa, capres, pilihan, nomor]"
4,prabowo kenapa harus sama gibran mau dukung an...,"[prabowo, kenapa, harus, sama, gibran, mau, du..."
...,...,...
1602,pilpres pilih prabowohatta karena tidak suka j...,"[pilpres, pilih, prabowohatta, karena, tidak, ..."
1603,infrastuktur pengolahan ikan hal yg penting un...,"[infrastuktur, pengolahan, ikan, hal, yg, pent..."
1604,anies baswedan berusaha menstabilkan dan menai...,"[anies, baswedan, berusaha, menstabilkan, dan,..."
1605,sorry ya lebih sexy kesalahan asam sulfat dari...,"[sorry, ya, lebih, sexy, kesalahan, asam, sulf..."


### Normalisasi

In [ ]:
key_norm = pd.read_excel('key_normalisasi.xlsx')
print(key_norm.head())

key_norm.shape

       singkat       hasil
0         abis       habis
1       accent     tekanan
2       accept      terima
3     accident  kecelakaan
4  achievement    prestasi


(3779, 2)

In [ ]:
normalizad_word = pd.read_excel("key_normalisasi.xlsx")
normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df4['normalisasi'] = df4['tokenize'].apply(normalized_term)

df4.loc[0:1607,['tokenize', 'normalisasi']]

,tokenize,normalisasi
0,"[iyaa, yg, ditanya, itu, lagi, itu, lagi, yg, ...","[iyaa, yang, ditanya, itu, lagi, itu, lagi, ya..."
1,"[si, marco, karundeng, di, kira, saya, hebat, ...","[si, marco, karundeng, di, kira, saya, hebat, ..."
2,"[seneng, lihat, pak, anies, seneng]","[senang, lihat, pak, anies, senang]"
3,"[ruarrr, biasa, capres, pilihan, nomor]","[ruarrr, biasa, capres, pilihan, nomor]"
4,"[prabowo, kenapa, harus, sama, gibran, mau, du...","[prabowo, kenapa, harus, sama, gibran, mau, du..."
...,...,...
1602,"[pilpres, pilih, prabowohatta, karena, tidak, ...","[pilpres, pilih, prabowohatta, karena, tidak, ..."
1603,"[infrastuktur, pengolahan, ikan, hal, yg, pent...","[infrastuktur, pengolahan, ikan, hal, yang, pe..."
1604,"[anies, baswedan, berusaha, menstabilkan, dan,...","[anies, baswedan, berusaha, menstabilkan, dan,..."
1605,"[sorry, ya, lebih, sexy, kesalahan, asam, sulf...","[maaf, ya, lebih, sexy, kesalahan, asam, sulfa..."


### Filtering (Stopword Removal)

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

list_stopwords = stopwords.words('indonesian')

In [ ]:
len(list_stopwords)

758

In [ ]:
list_stopwords[:10]

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir']

In [ ]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'boleh',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 'sekali',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'saja',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'begitu',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nggak',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah','gw','lu','lo','gtw','bukan',
                       'iyaa','si','ruarrr','itu','gue','dan','juga',
                       'cm','cmn','emg','mau','kali','jadi','begini',
                       'bila','nanti'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    #kata = [word for word in words if word not in list_stopwords]
    return [word for word in words if word not in list_stopwords]

df4['stopwords'] = df4['normalisasi'].apply(stopwords_removal)

df4.loc[0:1761,['normalisasi', 'stopwords']]

,normalisasi,stopwords
0,"[iyaa, yang, ditanya, itu, lagi, itu, lagi, ya...","[mengikuti, video, desak, anies, potongan, vid..."
1,"[si, marco, karundeng, di, kira, saya, hebat, ...","[marco, karundeng, hebat, ayam, sayur]"
2,"[senang, lihat, pak, anies, senang]","[senang, anies, senang]"
3,"[ruarrr, biasa, capres, pilihan, nomor]","[capres, pilihan, nomor]"
4,"[prabowo, kenapa, harus, sama, gibran, mau, du...","[prabowo, gibran, dukung, anies, wakil, cak, i..."
...,...,...
1602,"[pilpres, pilih, prabowohatta, karena, tidak, ...","[pilpres, pilih, prabowohatta, suka, jokowi, p..."
1603,"[infrastuktur, pengolahan, ikan, hal, yang, pe...","[infrastuktur, pengolahan, ikan, nelayan, perh..."
1604,"[anies, baswedan, berusaha, menstabilkan, dan,...","[anies, baswedan, berusaha, menstabilkan, mena..."
1605,"[maaf, ya, lebih, sexy, kesalahan, asam, sulfa...","[maaf, sexy, kesalahan, asam, sulfat, nyinyira..."


### Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd

# Create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stemmed wrapper function
def stemmed_wrapper(term):
    return stemmer.stem(term)

# Create a dictionary for unique terms
term_dict = {}
for document in df4['stopwords']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

# Apply stemming to unique terms
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

# Apply stemmed term to the DataFrame
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df4['stemming'] = df4['stopwords'].apply(get_stemmed_term)

df4.loc[0:1607,['stopwords', 'stemming']]

,stopwords,stemming
0,"[mengikuti, video, desak, anies, potongan, vid...","[ikut, video, desak, anies, potong, video, kla..."
1,"[marco, karundeng, hebat, ayam, sayur]","[marco, karundeng, hebat, ayam, sayur]"
2,"[senang, anies, senang]","[senang, anies, senang]"
3,"[capres, pilihan, nomor]","[capres, pilih, nomor]"
4,"[prabowo, gibran, dukung, anies, wakil, cak, i...","[prabowo, gibran, dukung, anies, wakil, cak, i..."
...,...,...
1602,"[pilpres, pilih, prabowohatta, suka, jokowi, p...","[pilpres, pilih, prabowohatta, suka, jokowi, p..."
1603,"[infrastuktur, pengolahan, ikan, nelayan, perh...","[infrastuktur, olah, ikan, nelayan, perhati, a..."
1604,"[anies, baswedan, berusaha, menstabilkan, mena...","[anies, baswedan, usaha, stabil, naik, dapat, ..."
1605,"[maaf, sexy, kesalahan, asam, sulfat, nyinyira...","[maaf, sexy, salah, asam, sulfat, nyinyir, ani..."


In [ ]:
# Assume 'stemming' column contains lists of stemmed words
df4['Clean'] = df4['stemming'].apply(lambda x: ' '.join(x))

# Display the resulting DataFrame
df4.head(10)

,tweet,tweet_clean,case_folding,tokenize,normalisasi,stopwords,stemming,Clean
0,@fairusizzudin @imanlagi Iyaa.. yg ditanya itu...,Iyaa yg ditanya itu lagi itu lagi yg nanya jg ...,iyaa yg ditanya itu lagi itu lagi yg nanya jg ...,"[iyaa, yg, ditanya, itu, lagi, itu, lagi, yg, ...","[iyaa, yang, ditanya, itu, lagi, itu, lagi, ya...","[mengikuti, video, desak, anies, potongan, vid...","[ikut, video, desak, anies, potong, video, kla...",ikut video desak anies potong video klasifikas...
1,Si Marco Karundeng di kira saya Hebat ternyata...,Si Marco Karundeng di kira saya Hebat ternyata...,si marco karundeng di kira saya hebat ternyata...,"[si, marco, karundeng, di, kira, saya, hebat, ...","[si, marco, karundeng, di, kira, saya, hebat, ...","[marco, karundeng, hebat, ayam, sayur]","[marco, karundeng, hebat, ayam, sayur]",marco karundeng hebat ayam sayur
2,@JamalBoegis @Abdulla55556212 Seneng lihat pak...,Seneng lihat pak Anies seneng,seneng lihat pak anies seneng,"[seneng, lihat, pak, anies, seneng]","[senang, lihat, pak, anies, senang]","[senang, anies, senang]","[senang, anies, senang]",senang anies senang
3,@aniesbaswedan Ruarrr biasa Capres pilihan Nom...,Ruarrr biasa Capres pilihan Nomor,ruarrr biasa capres pilihan nomor,"[ruarrr, biasa, capres, pilihan, nomor]","[ruarrr, biasa, capres, pilihan, nomor]","[capres, pilihan, nomor]","[capres, pilih, nomor]",capres pilih nomor
4,"prabowo kenapa harus sama gibran, mau dukung a...",prabowo kenapa harus sama gibran mau dukung an...,prabowo kenapa harus sama gibran mau dukung an...,"[prabowo, kenapa, harus, sama, gibran, mau, du...","[prabowo, kenapa, harus, sama, gibran, mau, du...","[prabowo, gibran, dukung, anies, wakil, cak, i...","[prabowo, gibran, dukung, anies, wakil, cak, i...",prabowo gibran dukung anies wakil cak imin hal...
5,Gue bilang the only hope karena emg diantara 3...,Gue bilang the only hope karena emg diantara p...,gue bilang the only hope karena emg diantara p...,"[gue, bilang, the, only, hope, karena, emg, di...","[saya, bilang, the, only, hope, karena, emang,...","[only, hope, emang, paslon, orientasinya, peme...","[only, hope, emang, paslon, orientasi, perata,...",only hope emang paslon orientasi perata transp...
6,@psi_id Anies Baswedan dan Gus Muhaimin menunj...,Anies Baswedan dan Gus Muhaimin menunjukkan ke...,anies baswedan dan gus muhaimin menunjukkan ke...,"[anies, baswedan, dan, gus, muhaimin, menunjuk...","[anies, baswedan, dan, gus, muhaimin, menunjuk...","[anies, baswedan, gus, muhaimin, kepemimpinan,...","[anies, baswedan, gus, muhaimin, pimpin, panda...",anies baswedan gus muhaimin pimpin pandai kelo...
7,Pak Anies : 👍🏻 Rektor Termuda Di Indonesia✅ Me...,Pak Anies Rektor Termuda Di Indonesia Menteri ...,pak anies rektor termuda di indonesia menteri ...,"[pak, anies, rektor, termuda, di, indonesia, m...","[pak, anies, rektor, termuda, di, indonesia, m...","[anies, rektor, termuda, indonesia, menteri, p...","[anies, rektor, muda, indonesia, menteri, didi...",anies rektor muda indonesia menteri didik jubi...
8,@tempodotco Tak bosan berharap Pak Presiden ne...,Tak bosan berharap Pak Presiden netral tidak m...,tak bosan berharap pak presiden netral tidak m...,"[tak, bosan, berharap, pak, presiden, netral, ...","[tidak, bosan, berharap, pak, presiden, netral...","[bosan, berharap, presiden, netral, main, aamiin]","[bosan, harap, presiden, netral, main, aamiin]",bosan harap presiden netral main aamiin
9,@DokterTifa @gibran_tweet Anies Baswedan dan G...,Anies Baswedan dan Gus Muhaimin teladan kepemi...,anies baswedan dan gus muhaimin teladan kepemi...,"[anies, baswedan, dan, gus, muhaimin, teladan,...","[anies, baswedan, dan, gus, muhaimin, teladan,...","[anies, baswedan, gus, muhaimin, teladan, kepe...","[anies, baswedan, gus, muhaimin, teladan, pimp...",anies baswedan gus muhaimin teladan pimpin hol...


In [ ]:
# Menyimpan DataFrame ke dalam file Excel
df4.to_excel('text_preprocessing_anies.xlsx', index=False)

In [ ]:
data_save = df4["Clean"]

In [ ]:
data_save

0       ikut video desak anies potong video klasifikas...
1                        marco karundeng hebat ayam sayur
2                                     senang anies senang
3                                      capres pilih nomor
4       prabowo gibran dukung anies wakil cak imin hal...
                              ...                        
1602    pilpres pilih prabowohatta suka jokowi pdi pil...
1603         infrastuktur olah ikan nelayan perhati anies
1604       anies baswedan usaha stabil naik dapat nelayan
1605    maaf sexy salah asam sulfat nyinyir anies tua ...
1606     anies presidenya nelayan rasa manfaat ide beliau
Name: Clean, Length: 1607, dtype: object

In [ ]:
data_save.to_csv('anies_clean_fix_2.csv')

In [ ]:
# Import libraries
import csv
import nltk
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemover, ArrayDictionary
import string
import os

In [ ]:
#load dataset
df5 = pd.read_csv('anies_clean_fix_2.csv')
df5

,Unnamed: 0,Clean
0,0,ikut video desak anies potong video klasifikas...
1,1,marco karundeng hebat ayam sayur
2,2,senang anies senang
3,3,capres pilih nomor
4,4,prabowo gibran dukung anies wakil cak imin hal...
...,...,...
1602,1602,pilpres pilih prabowohatta suka jokowi pdi pil...
1603,1603,infrastuktur olah ikan nelayan perhati anies
1604,1604,anies baswedan usaha stabil naik dapat nelayan
1605,1605,maaf sexy salah asam sulfat nyinyir anies tua ...


In [ ]:
df5 = df5.drop(columns=['Unnamed: 0'])
df5

,Clean
0,ikut video desak anies potong video klasifikas...
1,marco karundeng hebat ayam sayur
2,senang anies senang
3,capres pilih nomor
4,prabowo gibran dukung anies wakil cak imin hal...
...,...
1602,pilpres pilih prabowohatta suka jokowi pdi pil...
1603,infrastuktur olah ikan nelayan perhati anies
1604,anies baswedan usaha stabil naik dapat nelayan
1605,maaf sexy salah asam sulfat nyinyir anies tua ...


In [ ]:
df5.columns

Index(['Clean'], dtype='object')

# Labelisasi Data

In [ ]:
lexicon_positif_dict = {}
with open('positive.tsv', 'r', newline='', encoding='utf-8') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')

    # Skip header
    next(reader, None)

    for row in reader:
        if row[0] not in lexicon_positif_dict:
            lexicon_positif_dict[row[0]] = int(row[1])

lexicon_negatif_dict = {}
with open('negative.tsv', 'r', newline='', encoding='utf-8') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')

    # Skip header
    next(reader, None)

    for row in reader:
        if row[0] not in lexicon_negatif_dict:
            lexicon_negatif_dict[row[0]] = int(row[1])

def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text:
        if word in lexicon_positif_dict:
            score += lexicon_positif_dict[word]
        if word in lexicon_negatif_dict:
            score += lexicon_negatif_dict[word]
    sentimen = 'Positif' if score >= 0 else 'Negatif'
    return score, sentimen

In [ ]:
df5

,Clean
0,ikut video desak anies potong video klasifikas...
1,marco karundeng hebat ayam sayur
2,senang anies senang
3,capres pilih nomor
4,prabowo gibran dukung anies wakil cak imin hal...
...,...
1602,pilpres pilih prabowohatta suka jokowi pdi pil...
1603,infrastuktur olah ikan nelayan perhati anies
1604,anies baswedan usaha stabil naik dapat nelayan
1605,maaf sexy salah asam sulfat nyinyir anies tua ...


In [ ]:
df5['Tokenized 2'] = df5['Clean'].apply(lambda x: word_tokenize_wrapper(str(x)) if pd.notna(x) else '')

In [ ]:
df5

,Clean,Tokenized 2
0,ikut video desak anies potong video klasifikas...,"[ikut, video, desak, anies, potong, video, kla..."
1,marco karundeng hebat ayam sayur,"[marco, karundeng, hebat, ayam, sayur]"
2,senang anies senang,"[senang, anies, senang]"
3,capres pilih nomor,"[capres, pilih, nomor]"
4,prabowo gibran dukung anies wakil cak imin hal...,"[prabowo, gibran, dukung, anies, wakil, cak, i..."
...,...,...
1602,pilpres pilih prabowohatta suka jokowi pdi pil...,"[pilpres, pilih, prabowohatta, suka, jokowi, p..."
1603,infrastuktur olah ikan nelayan perhati anies,"[infrastuktur, olah, ikan, nelayan, perhati, a..."
1604,anies baswedan usaha stabil naik dapat nelayan,"[anies, baswedan, usaha, stabil, naik, dapat, ..."
1605,maaf sexy salah asam sulfat nyinyir anies tua ...,"[maaf, sexy, salah, asam, sulfat, nyinyir, ani..."


In [ ]:
# Perform sentiment analysis using the lexicon
results = df5['Tokenized 2'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df5['Polarity Score'] = results[0]
df5['polarity'] = results[1]

# Display relevant columns
print(df5['polarity'].value_counts()),df5.shape

polarity
Positif    810
Negatif    797
Name: count, dtype: int64


(None, (1607, 4))

In [ ]:
df5.to_excel("Data Labelling Anies_2.xlsx")